## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yining,CN,29.04,114.56,74.46,82,0,1.19,clear sky
1,1,Rikitea,PF,-23.12,-134.97,69.40,84,94,13.62,light rain
2,2,Marsa Matruh,EG,31.35,27.25,82.40,74,0,9.17,clear sky
3,3,Bluff,NZ,-46.60,168.33,45.00,72,99,11.01,overcast clouds
4,4,Tasiilaq,GL,65.61,-37.64,41.00,75,100,14.99,light intensity shower rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yining,CN,29.04,114.56,74.46,82,0,1.19,clear sky
2,2,Marsa Matruh,EG,31.35,27.25,82.40,74,0,9.17,clear sky
6,6,Ribeira Grande,PT,38.52,-28.70,78.80,65,20,6.93,few clouds
7,7,Kutum,SD,14.20,24.67,81.88,47,51,5.73,broken clouds
9,9,Guerrero Negro,MX,27.98,-114.06,75.15,62,0,6.89,clear sky
10,10,Albany,US,42.60,-73.97,70.00,62,51,3.27,broken clouds
15,15,San Patricio,MX,19.22,-104.70,82.40,74,75,4.45,broken clouds
16,16,Maun,BW,-19.98,23.42,80.60,18,0,6.93,clear sky
17,17,Vaini,TO,-21.20,-175.20,75.20,94,75,9.17,broken clouds
18,18,Castro,BR,-24.79,-50.01,74.10,60,12,8.05,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yining,CN,74.46,clear sky,29.04,114.56,
2,Marsa Matruh,EG,82.40,clear sky,31.35,27.25,
6,Ribeira Grande,PT,78.80,few clouds,38.52,-28.70,
7,Kutum,SD,81.88,broken clouds,14.20,24.67,
9,Guerrero Negro,MX,75.15,clear sky,27.98,-114.06,
10,Albany,US,70.00,broken clouds,42.60,-73.97,
15,San Patricio,MX,82.40,broken clouds,19.22,-104.70,
16,Maun,BW,80.60,clear sky,-19.98,23.42,
17,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
18,Castro,BR,74.10,few clouds,-24.79,-50.01,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True) 
print(hotel_df)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
print(hotel_df)

               City Country  Max Temp Current Description    Lat     Lng  \
0            Yining      CN     74.46           clear sky  29.04  114.56   
2      Marsa Matruh      EG     82.40           clear sky  31.35   27.25   
6    Ribeira Grande      PT     78.80          few clouds  38.52  -28.70   
7             Kutum      SD     81.88       broken clouds  14.20   24.67   
9    Guerrero Negro      MX     75.15           clear sky  27.98 -114.06   
..              ...     ...       ...                 ...    ...     ...   
693       San-Pedro      CI     76.35     overcast clouds   4.75   -6.64   
695          Bonoua      CI     80.60          few clouds   5.27   -3.60   
696     Kitaibaraki      JP     73.99       broken clouds  36.78  140.75   
698     Zhanakorgan      KZ     81.50     overcast clouds  43.91   67.25   
700        Boralday      KZ     75.20       broken clouds  43.36   76.86   

                       Hotel Name  
0                   Xiushui Hotel  
2              

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))